In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
import  math

from sklearn.feature_selection import RFE
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from matplotlib import rcParams
from matplotlib import pyplot
from xgboost import plot_importance
from numpy import sort
from numpy import loadtxt
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
%matplotlib inline
from sklearn.model_selection import GridSearchCV
import pickle

In [34]:
# evaluate AnnotSV

In [8]:
cnv_types = ['loss', 'gain']
cnv_type = cnv_types[1]

x_test_f = './data_annotsv/train_val_test_sets/train_val_test/x_test_' + cnv_type + '.tsv'
y_test_f = './data_annotsv/train_val_test_sets/train_val_test/y_test_'+ cnv_type + '.tsv'

x_test = pd.read_csv(x_test_f, sep='\t')
y_test = pd.read_csv(y_test_f, sep='\t')


In [9]:
def get_category(row):
    if row.clinsig_bin == 0:
        if row['AnnotSV ranking'] == 1 or row['AnnotSV ranking'] == 2:
            category = 'TN'
        elif row['AnnotSV ranking'] == 4 or row['AnnotSV ranking'] == 5:
            category = 'FP'
        else:
            category = 'uncertain'
    else:
        if row['AnnotSV ranking'] == 1 or row['AnnotSV ranking'] == 2:
            category = 'FN'
        elif row['AnnotSV ranking'] == 4 or row['AnnotSV ranking'] == 5:
            category = 'TP'
        else:
            category = 'uncertain'
    return category

In [10]:
cnv_types = ['loss', 'gain']
all_acc = pd.DataFrame(columns=['cnv_type', 'Accuracy', 'FP','TN','TP','FN', 'uncertain'])

for cnv_type in cnv_types:
    x_test_f = './data_annotsv/train_val_test_sets/train_val_test/x_test_' + cnv_type + '.tsv'
    y_test_f = './data_annotsv/train_val_test_sets/train_val_test/y_test_'+ cnv_type + '.tsv'

    x_test = pd.read_csv(x_test_f, sep='\t')
    y_test = pd.read_csv(y_test_f, sep='\t')
    
    df = x_test[['AnnotSV ranking', 'clinsig_bin']]
    df['Category_AnnotSV'] = df.apply(get_category, axis=1)
    
    x = pd.DataFrame([[cnv_type]], columns=['cnv_type'])

    TrueFalse_category = pd.DataFrame.from_dict(Counter(df.Category_AnnotSV), orient='index').transpose()
    TN = TrueFalse_category.iloc[0] ['TN']
    FN = TrueFalse_category.iloc[0] ['FN']
    TP = TrueFalse_category.iloc[0] ['TP']
    FP = TrueFalse_category.iloc[0] ['FP']
#     uncl = TrueFalse_category.iloc[0] ['uncertain'] # netreba, nieje ziadne uncertain v testovacom sete
    print(TN, FN, TP, FP)
    print('Accuracy:',cnv_type, (TP + TN)/(TP + TN + FP + FN))
    accuracy = ((TP + TN)/(TP + TN + FP + FN))

    c = pd.DataFrame([[accuracy]], columns=['Accuracy'])
    x = pd.concat([x, c, TrueFalse_category], axis=1, ignore_index=False)
    all_acc = all_acc.append(x)
all_acc

/home/gaziova/miniconda3/envs/notebookProjects/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


959 1 890 676
Accuracy: loss 0.731987331749802
755 3 425 625
Accuracy: gain 0.6526548672566371


,cnv_type,Accuracy,FP,TN,TP,FN,uncertain
0,loss,0.731987,676,959,890,1,NaN
0,gain,0.652655,625,755,425,3,NaN


In [35]:
# evaluate ClassifyCNV

In [92]:
def get_category_ClassifyCNV(row):
    if row.clinsig_bin == 0:
        if row.Classification == 'Benign' or row.Classification == 'Likely benign':
            category = 'TN'
        elif row.Classification == 'Likely pathogenic' or row.Classification == 'Pathogenic':
            category = 'FP'
        else:
            category = 'uncertain'
    else:
        if row.Classification == 'Benign' or row.Classification == 'Likely benign':
            category = 'FN'
        elif row.Classification == 'Likely pathogenic' or row.Classification == 'Pathogenic':
            category = 'TP'
        else:
            category = 'uncertain'
    return category


In [95]:
cnv_types = ['loss', 'gain']
all_acc = pd.DataFrame(columns=['cnv_type', 'Accuracy', 'FP','TN','TP','FN', 'uncertain'])

for cnv_type in cnv_types:
    result_fn = '../classifyCNV_test/cnv_clinvar/ClassifyCNV/ClassifyCNV_results/classifyCNV_results_cnv_isv_annotsv_testdata/test_'+ cnv_type +'_isv_annotsv/Scoresheet.txt'
    df_result = pd.read_csv(result_fn, sep='\t', low_memory=False)
    df_result = df_result[['Chromosome', 'Start', 'End', 'Classification','Total score']]
    df_result.columns = ['Chromosome', 'SV_start', 'SV_end', 'Classification','Total_score']
    df_result['chrom'] = df_result['Chromosome'].str.replace('chr', '')
    
    x_test_f = './data_annotsv/train_val_test_sets/train_val_test/x_test_' + cnv_type + '.tsv'
    x_test = pd.read_csv(x_test_f, sep='\t')
    test_set = x_test[['chrom', 'SV_start', 'SV_end', 'clinsig_bin', 'rcvAcc']]
    test_set_merged = test_set.merge(df_result, how='inner', on=['chrom', 'SV_start', 'SV_end'])
    test_set_merged['Category_ClassifyCNV'] = test_set_merged.apply(get_category_ClassifyCNV, axis=1)
    
    x = pd.DataFrame([[cnv_type]], columns=['cnv_type'])

    TrueFalse_category = pd.DataFrame.from_dict(Counter(test_set_merged.Category_ClassifyCNV), orient='index').transpose()
    TN = TrueFalse_category.iloc[0] ['TN']
    if 'FN' in list(TrueFalse_category.columns):
        FN = TrueFalse_category.iloc[0] ['FN']
    else:
        FN = 0
    TP = TrueFalse_category.iloc[0] ['TP']
    if 'FP' in list(TrueFalse_category.columns):
        FP = TrueFalse_category.iloc[0] ['FP']
    else:
        FP = 0
    uncl = TrueFalse_category.iloc[0] ['uncertain']
    print(TN, FN, TP, FP)
    print('Accuracy:',cnv_type, (TP + TN)/(TP + TN + FP + FN))
    accuracy = ((TP + TN)/(TP + TN + FP + FN))

    c = pd.DataFrame([[accuracy]], columns=['Accuracy'])
    x = pd.concat([x, c, TrueFalse_category], axis=1, ignore_index=False)
    all_acc = all_acc.append(x)
all_acc

229 1 547 9
Accuracy: loss 0.9872773536895675
196 0 285 3
Accuracy: gain 0.993801652892562


,cnv_type,Accuracy,FP,TN,TP,FN,uncertain
0,loss,0.987277,9,229,547,1,1740
0,gain,0.993802,3,196,285,NaN,1324


In [5]:
132400/1808

73.23008849557522